In [1]:
import fasttext

In [2]:
model = fasttext.load_model("amazon_review_full.bin")

In [67]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        coefs = np.asarray(tokens[1:], dtype='float32')
        data[tokens[0]] = coefs
    
    return data

In [68]:
data = load_vectors('./wiki-news-300d-1M-subword.vec')

In [78]:
DIM = 300

##### sentence-similarity

In [133]:
s1 = "Salt Water Sandals by Hoy Original Sandal (Baby, Walker, Toddler, Little Kid & Big Kid) SHINY YELLOW Little Kid 2 M"
s2 = "Salt Water Sandals by Hoy Shoe The Original Sandal Shiny Yellow 1 Little Kid"

In [139]:
e1 = sen_emb(s1) 
e2 = sen_emb(s2)

In [140]:
e1, e2 = pad_emb(e1, e2, DIM)

In [141]:
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

0.20634976781033643


In [142]:
e1_ = avg_emb(s1, DIM)
e2_ = avg_emb(s2, DIM)

In [138]:
cos_sim = np.dot(e1_, e2_)/(norm(e1_)*norm(e2_))
print(cos_sim)

1.0


##### word-similarity

In [154]:
e1 = data['mask']
e2 = data['health']
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

0.34597263


##### alpha-numeric

In [164]:
s1 = "2M size shirt"
s2 = "shirt"

e1 = sen_emb(s1) 
e2 = sen_emb(s2)

e1, e2 = pad_emb(e1, e2, DIM)

cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

0.045931146871255435


##### spell-mistakes

In [ ]:
e1 = data['laptop']
e2 = data['lapttp']
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

##### prefix / suffix

In [182]:
e1 = data['clean']
e2 = data['cleaner']
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

0.7213481


##### singular / plural 

In [186]:
e1 = data['shoe']
e2 = data['shoes']
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

0.77948236


##### Context check

In [194]:
s1 = "river bank holds surprises for people"
s2 = "Federal bank holds surprise for people"

e1 = sen_emb(s1) 
e2 = sen_emb(s2)

e1, e2 = pad_emb(e1, e2, DIM)

cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

0.9055997742572052


##### Phrases detection 

In [211]:
e1 = data['newyork']
e2 = data['washington']
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

0.57993096


#### amazon_exp

In [223]:
DIM = 10

In [232]:
s1 = "Jo Malone London™ Dark Amber & Ginger Lily Scented Home Candle NO COLOR  7 oz"
s2 = "Jo Malone London Dark Amber & Ginger Lily Home Candle"

e1 = sen_emb(s1) 
e2 = sen_emb(s2)

e1, e2 = pad_emb(e1, e2, DIM)

cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

e1_ = avg_emb(s1, DIM)
e2_ = avg_emb(s2, DIM)

cos_sim = np.dot(e1_, e2_)/(norm(e1_)*norm(e2_))
print(cos_sim)

0.7039840221103811
-0.14641275


In [242]:
e1 = model['see']
e2 = model['sea']
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

0.20315863


In [252]:
s1 = "weterproof"
s2 = "water proof"

e1 = sen_emb(s1) 
e2 = sen_emb(s2)

e1, e2 = pad_emb(e1, e2, DIM)

cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

nan


C:\Users\Akshay\.conda\envs\pytorchenv\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [253]:
e1 = model['jecket']
e2 = model['jacket']
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

-0.004059964


In [267]:
s1 = "river bank holds surprises for people"
s2 = "Federal bank holds surprise for people"

e1 = sen_emb(s1) 
e2 = sen_emb(s2)

e1, e2 = pad_emb(e1, e2, DIM)

cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim) 

e1_ = avg_emb(s1, DIM)
e2_ = avg_emb(s2, DIM)

cos_sim = np.dot(e1_, e2_)/(norm(e1_)*norm(e2_))
print(cos_sim)

0.8180187902965685
0.924704


In [276]:
e1 = model['waterproof']
e2 = model['resistant']
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2))
print(cos_sim)

0.6297622


In [219]:
PUNCT_TO_REMOVE = string.punctuation
PUNCT_TO_REMOVE = PUNCT_TO_REMOVE.replace("-","")
PUNCT_TO_REMOVE = PUNCT_TO_REMOVE.replace(".","")
PUNCT_TO_REMOVE = PUNCT_TO_REMOVE.replace("&","")

def remove_special_chars(text):
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))


def sen_emb(sen):
    # pre-process the sentence
    sen = sen.replace('-',' ')
    sen = sen.replace('/',' ')
    sen = remove_special_chars(sen)
    sen = sen.replace('&','and')
    sen = sen.lower()
    
    ## split the sen to list of words
    words = sen.split()
    
    emb = []
    emb = np.array(emb)
        

    for word in words:
        try:
            vec = model[word]
        
        except KeyError:
            continue
    
        emb = np.concatenate([emb, vec])
        
    return emb    
    

In [218]:
def pad_emb(e1, e2, dim):
    s_ = e1.shape
    size_1 = s_[0]
    
    s_ = e2.shape
    size_2 = s_[0]
    
    if size_1 == size_2:
        return [e1, e2]
    
    elif size_1 < size_2:
        for _ in range(size_2 - size_1):
            e1 = np.append(e1, 0.0)
        
    elif size_1 > size_2:
        for _ in range(size_1 - size_2):
            e2 = np.append(e2, 0.0)
            
    return [e1, e2]

In [217]:
def avg_emb(sen, dim):
    sen = sen.replace('-',' ')
    sen = sen.replace('/',' ')
    sen = remove_special_chars(sen)
    sen = sen.replace('&','and')
    sen = sen.lower()
    words = sen.split()
    
    vec = np.zeros(dim)
    
    for word in words:
        try:
            v = model[word]
        except KeyError:
            continue
            
        v.setflags(write=1) 
        vec.setflags(write=1)
        
        if vec[0] == 0:
            vec = v
            
        for i in range(dim):
            vec[i] = (v[i] + vec[i])/2
            
    return vec   

In [77]:
len(model.words)

1402671

In [55]:
e1 = model['book']

In [56]:
e2 = model['books']

In [76]:
cos_sim = np.dot(e1, e2)/(norm(e1)*norm(e2)) 
print(cos_sim)

0.35967568


In [16]:
model.get_analogies('water', 'what', 'cold')

[(0.9681640267372131, 'mostrandose'),
 (0.9668797254562378, 'man-up'),
 (0.9656786322593689, 'consalmost'),
 (0.9618635177612305, 'onlycomplaint'),
 (0.9574946165084839, 'smartshot'),
 (0.9567878246307373, 'magazine-late'),
 (0.9553950428962708, 'monorails'),
 (0.9535049200057983, 'fantadtic'),
 (0.9516035318374634, 'thorough--it'),
 (0.951547384262085, 'a+/server+/net+')]

In [17]:
'narc' in model

True

In [20]:
model['rambo']

array([-0.07588956, -0.13569047,  0.19736196, -0.09127723,  0.19365077,
        0.03731917,  0.1109871 , -0.06580413, -0.09331769,  0.22891137],
      dtype=float32)

In [23]:
model.get_nearest_neighbors('water')

[(0.9841490983963013, 'non-stimulating'),
 (0.9620241522789001, 'noexplanation'),
 (0.959243893623352, 'shifting-of-gears'),
 (0.9591127634048462, 'ehte'),
 (0.9519392848014832, 'reccomend=-'),
 (0.950046956539154, 'munchkinland'),
 (0.9480462074279785, 'mrf'),
 (0.9475089907646179, 'onda'),
 (0.9469833970069885, 'gorim'),
 (0.9462774395942688, 'absoluteally')]